# A known analytical solution
Author: Jørgen S. Dokken

Just as for the [Poisson problem](./../chapter1/fundamentals_code), we construct a test problem which makes it easy to determine if the calculations are correct.

Since we know that our first-order time-stepping scheme is exact for linear functions, we create a problem which has linear variation in time. We combine this with a quadratic variation in space. Therefore, we choose the analytical solution to be
\begin{align}
u = 1 + x^2+\alpha y^2 + \beta t
\end{align}
which yields a function whose computed values at the degrees of freedom will be exact, regardless of the mesh size and $\Delta t$ as long as the mesh is uniformly partitioned.
By inserting this into our original PDE, we find that the right hand side $f=\beta-2-2\alpha$. The boundary value $u_d(x,y,t)=1+x^2+\alpha y^2 + \beta t$ and the initial value $u_0(x,y)=1+x^2+\alpha y^2$.

We start by defining the temporal discretization parameters, along with the parameters for $\alpha$ and $\beta$.

In [1]:
t = 0 # Start time
T = 2 # End time
num_steps = 20 # Number of time steps
dt = (T-t)/num_steps # Time step size
alpha = 3
beta = 1.2

As for the previous problem, we define the mesh and appropriate function spaces.

In [2]:
import dolfinx
import numpy
from dolfinx.cpp.mesh import CellType
from mpi4py import MPI
from petsc4py import PETSc

nx, ny = 5, 5
mesh = dolfinx.UnitSquareMesh(MPI.COMM_WORLD, nx, ny, CellType.triangle)
V = dolfinx.FunctionSpace(mesh, ("CG", 1))

## Defining the exact solution
As in the membrane problem, we create a Python-class to resemble the exact solution

In [3]:
class exact_solution():
    def __init__(self, alpha, beta, t):
        self.alpha = alpha
        self.beta = beta
        self.t = t
    def __call__(self, x):
        return 1 + x[0]**2 + self.alpha * x[1]**2 + self.beta * self.t
u_exact = exact_solution(alpha, beta, t)

## Defining the boundary condition
As in the previous chapters, we define a Dirichlet boundary condition over the whole boundary

In [4]:
u_D = dolfinx.Function(V)
u_D.interpolate(u_exact)
u_D.vector.ghostUpdate(addv=PETSc.InsertMode.INSERT, mode=PETSc.ScatterMode.FORWARD)
fdim = mesh.topology.dim - 1
boundary_facets = dolfinx.mesh.locate_entities_boundary(
    mesh, fdim, lambda x: numpy.full(x.shape[1], True, dtype=bool))
bc = dolfinx.DirichletBC(u_D, dolfinx.fem.locate_dofs_topological(V, fdim, boundary_facets))

## Defining the variational formualation
As we have set $t=0$ in `u_exact`, we can reuse this variable to obtain $u_n$ for the first time step.

In [5]:
u_n = dolfinx.Function(V)
u_n.interpolate(u_exact)
u_n.vector.ghostUpdate(addv=PETSc.InsertMode.INSERT, mode=PETSc.ScatterMode.FORWARD)

As $f$ is a constant independent of $t$, we can define it as a constant.

In [6]:
f = dolfinx.Constant(mesh, beta - 2 - 2 * alpha)

We can now create our variational formulation, with the bilinear form `a` and  linear form `L`.

In [7]:
import ufl
u, v = ufl.TrialFunction(V), ufl.TestFunction(V)
F = u*v*ufl.dx + dt*ufl.dot(ufl.grad(u), ufl.grad(v))*ufl.dx - (u_n + dt*f)*v*ufl.dx
a = ufl.lhs(F)
L = ufl.rhs(F)

## Create the matrix and and vector for the linear problem
To ensure that we are solving the variational problem efficiently, we will create several structures which can reuse data, such as matrix sparisty patterns. Especially note as the bilinear form `a` is independent of time, we only need to assemble the matrix once.

In [8]:
A = dolfinx.fem.assemble_matrix(a, bcs=[bc])
A.assemble()
b = dolfinx.fem.create_vector(L)
uh = dolfinx.Function(V)

## Define a linear variational solver
We will use [PETSc](https://www.mcs.anl.gov/petsc/) to solve the resulting linear algebra problem. We use the Python-API `petsc4py` to define the solver. We will use a linear solver.

In [9]:
solver = PETSc.KSP().create(mesh.mpi_comm())
solver.setOperators(A)
solver.setType(PETSc.KSP.Type.PREONLY)
solver.getPC().setType(PETSc.PC.Type.LU)

## Solving the time-dependent problem
With these structures in place, we crete our time-stepping loop.
In this loop, we first update the Dirichlet boundary condition by interpolating the updated 
expression `u_exact` into `V`. The next step is to re-assemble the vector `b`, with the update `u_n`.
Then, we need to apply the boundary condition to this vector. We do this by using the lifting operation,
which applies the boundary condition such that symmetry of the matrix is preserved.
Then we solve the problem using PETSc and update `u_n` with the data from `uh`.

In [10]:
for n in range(num_steps):
    # Update Diriclet boundary condition 
    u_exact.t+=dt
    u_D.interpolate(u_exact)
    u_D.vector.ghostUpdate(addv=PETSc.InsertMode.INSERT, mode=PETSc.ScatterMode.FORWARD)
    
    # Update the right hand side reusing the initial vector
    with b.localForm() as loc_b:
        loc_b.set(0)
    dolfinx.fem.assemble_vector(b, L)
    
    # Apply Dirichlet boundary condition to the vector
    dolfinx.fem.apply_lifting(b, [a], [[bc]])
    b.ghostUpdate(addv=PETSc.InsertMode.ADD_VALUES, mode=PETSc.ScatterMode.REVERSE)
    dolfinx.fem.set_bc(b, [bc])

    # Solve linear problem
    solver.solve(b, uh.vector)
    uh.vector.ghostUpdate(addv=PETSc.InsertMode.INSERT_VALUES, mode=PETSc.ScatterMode.FORWARD)    
    # Update solution at previous time step (u_n)
    with uh.vector.localForm() as loc, u_n.vector.localForm() as loc_n:
        loc.copy(loc_n)

## Verifying the numerical solution
As in the first chapter, we compute the L2-error and the error at the mesh vertices 
to verify our implementation.

In [11]:
# Compute L2 error and error at nodes
V_ex = dolfinx.FunctionSpace(mesh, ("CG", 2))
u_ex = dolfinx.Function(V_ex)
u_ex.interpolate(u_exact)
u_ex.vector.ghostUpdate(addv=PETSc.InsertMode.INSERT, mode=PETSc.ScatterMode.FORWARD)
error_L2 = numpy.sqrt(MPI.COMM_WORLD.allreduce(dolfinx.fem.assemble_scalar((uh - u_ex)**2 * ufl.dx), op=MPI.SUM))
print(f"L2-error: {error_L2:.2e}")

# Compute values at mesh vertices
u_vertex_values = uh.compute_point_values()
u_ex_vertex_values = u_ex.compute_point_values()
error_max = numpy.max(numpy.abs(u_vertex_values - u_ex_vertex_values))
print(f"Error_max: {error_max:.2e}")

L2-error: 2.83e-02
Error_max: 7.11e-15
